In [0]:
# Dummy code to get more RAM on google colab
#a = []
#while(1):
 #   a.append('1')

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import pandas as pd
import wave
import sys
import os
import librosa
import librosa.display
import xgboost as xgb
from  sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import sklearn.naive_bayes as nb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier,GradientBoostingClassifier
from sklearn.neighbors import kd_tree
import seaborn as sn
from sklearn.metrics import confusion_matrix
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [0]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint


from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [0]:
# Get per cycle data frame
# Load per cycle data frame
import pickle
folder = "/content/drive/My Drive/Respiratory_Sound_Database/Respiratory_Sound_Database/" 
filename = folder + 'w_c_dataset.pickle'
infile = open(filename,'rb')
[sound,sr,lengths,times,labels] = pickle.load(infile)
infile.close()

In [0]:
import sys

def split_sounds(sounds,times,labels):
    s=[]
    l=[]
    for i,sound in enumerate(sounds):
        for t,label in zip(times[i],labels[i]):
            s.append(sound[int(t[0]):int(t[1])])
            if label==0:
                a=np.array([1,0,0,0])
            if label==1:
                a=np.array([0,1,0,0])
            if label==2:
                a=np.array([0,0,1,0])    
            if label==3:
                a=np.array([0,0,0,1])
            l.append(a)
    return s,l

[data,label]=split_sounds(sound,times,labels)

In [0]:
# Get data from CWT Feature picke
folder = "/content/drive/My Drive/Respiratory_Sound_Database/Respiratory_Sound_Database/" 
filename = folder + 'wavelet.pickle'
infile = open(filename,'rb')
wave_dict = pickle.load(infile)
infile.close()

In [0]:
dataset = wave_dict['wavelet']

In [20]:
# add channel dimension for CNN
dataset = np.reshape(dataset, (*dataset.shape,1))

label=np.asarray(label)
a=np.zeros(label.shape[0])
for i in range(label.shape[0]):
    for j in range(label.shape[1]):
        if label[i][j]==1:
            a[i]=j

# COnvert to binary for the binary classification problem
bin_label =np.zeros(label.shape[0])
for index,l in enumerate(a):
    if l == 0:
        bin_label[index] = l
    else:
        bin_label[index] = 1

unique, counts = np.unique(bin_label, return_counts=True)

print(np.asarray((unique, counts)).T)


# One-hot encode labels
le = LabelEncoder()
i_labels = le.fit_transform(bin_label)
oh_labels = to_categorical(i_labels)

[[0.000e+00 3.642e+03]
 [1.000e+00 3.256e+03]]


In [0]:
# train test split
x_train, x_test, y_train, y_test = train_test_split(dataset, oh_labels, stratify=oh_labels, 
                                                    test_size=0.2, random_state = 42)

In [0]:
num_rows = 40
num_columns = 697
num_channels = 1

num_labels = 2
filter_size = 2

# Construct model 
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=filter_size,
                 input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=filter_size, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=filter_size, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=filter_size, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(GlobalAveragePooling2D())

model.add(Dense(32, activation='relu')) 

model.add(Dense(num_labels, activation='sigmoid')) 

In [0]:
# Compile the model
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam') 

In [24]:
# Display model architecture summary 
model.summary()



# Load the trained model if required
#model_file = '/content/drive/My Drive/Respiratory_Sound_Database/Respiratory_Sound_Database/FinalModels/MFCC_CNN_Binary/MFC_CNN_Binary_197.h5'
#model.load_weights(model_file)

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 39, 696, 16)       80        
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 19, 348, 16)       0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 19, 348, 16)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 18, 347, 32)       2080      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 9, 173, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 9, 173, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 8, 172, 64)       

In [25]:
# train model
num_epochs = 150
num_batch_size = 128



callbacks = [
    ModelCheckpoint(
        filepath='/content/drive/My Drive/Respiratory_Sound_Database/Respiratory_Sound_Database/FinalModels/CWT_CNN/Try2/CWT_CNN_Binary_{epoch:02d}.h5',
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_accuracy` score has improved.
        save_best_only=True,
        monitor='val_accuracy',
        verbose=1)
]
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs,
          validation_data=(x_test, y_test), callbacks=callbacks, verbose=1)

duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/150
43/44 [============================>.] - ETA: 0s - loss: 0.6910 - accuracy: 0.5649
Epoch 00001: val_accuracy improved from -inf to 0.59275, saving model to /content/drive/My Drive/Respiratory_Sound_Database/Respiratory_Sound_Database/FinalModels/CWT_CNN/Try2/CWT_CNN_Binary_01.h5
44/44 [==============================] - 2s 45ms/step - loss: 0.6904 - accuracy: 0.5649 - val_loss: 0.6780 - val_accuracy: 0.5928
Epoch 2/150
43/44 [============================>.] - ETA: 0s - loss: 0.6751 - accuracy: 0.5905
Epoch 00002: val_accuracy improved from 0.59275 to 0.59565, saving model to /content/drive/My Drive/Respiratory_Sound_Database/Respiratory_Sound_Database/FinalModels/CWT_CNN/Try2/CWT_CNN_Binary_02.h5
44/44 [==============================] - 2s 44ms/step - loss: 0.6753 - accuracy: 0.5903 - val_loss: 0.6777 - val_accuracy: 0.5957
Epoch 3/150
43/44 [============================>.] - ETA: 0s - loss: 0.6631 - accuracy: 0.6248
Epoch 00003: val_accuracy improved from 0.59565 to 0.61014